# Let's train a classifier

In [1]:
# 1. Look up structure
# 2. Load x_train, extract y_train
# 3. Load x_val/x_test, create/extract y_val/y_test
# 4. write down classifier
# 5. train classifier
# 6. evaluate classifier with test_data

In [2]:
import os
# os.system("pip install pandas")
# os.system("pip install torchvision")
os.system("CUDA_LAUNCH_BLOCKING=1")
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms
from torchvision.models import resnet101, resnet34, ResNet101_Weights, ResNet34_Weights

import time

import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

In [3]:
# set paths and constants
attributes_path = "datasets/celeba/list_attr_celeba.txt"
partitions_path = 'datasets/celeba/list_eval_partition.txt'
race_path = "CelebA/races/races_ff.csv"
label_dir = "CelebA/labels_split/"
img_celeb_dir = "CelebA/cropped/"
img_created_dir = "CelebA/augmented/raw_input_auto_gender_cropped"
train_data_path = ""

size_train = 500
size_val = 1000
use_baseline = False
images_per_image_id = 1 if use_baseline else 7

# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 9e-5 if use_baseline else 7e-5
NUM_EPOCHS = 10

# Architecture
FEATURE_SIZE = (512, 512)
BATCH_SIZE_TRAIN = 128
BATCH_SIZE_VAL = 128
BATCH_SIZE_TEST = 128
DEVICE = 'cuda:0'


races = ["Black", "Indian", "Latino", "Middle Eastern", "Southeast Asian", "East Asian", "White"]

In [4]:
# load all labels

# all_labels = pd.read_csv(attributes_path, sep="\s+", skiprows=1).rename_axis("Image_Name").reset_index().replace(-1, 0)
# print(all_labels.head())

# # create training and validation labels out off all_labels
# y_train = all_labels[:size_train]
# # y_train.columns = all_labels.columns
# print(y_train.shape)

# y_train = pd.DataFrame(columns=all_labels.columns)
# for i, row in tqdm(all_labels[:size_train].iterrows(), total=size_train):
#     img_number = f"{(i+1):06}"
#     for j, race in enumerate(races):
#         new_image_name = f"{img_number}_{j}_{race.lower()}.jpg"
#         row[0] = new_image_name
#         y_train = y_train.append(row)
#     # if i == 0:
#     #     break
# print(y_train.shape)
# y_train.to_csv(label_dir + "created_train2.csv")

# y_val = all_labels[-1000:]
# y_val.to_csv(label_dir + "val.csv")

# y_test = all_labels[-2000:-1000]
# y_test.to_csv(label_dir + "test.csv")

# print("y_train shape:", y_train.shape)
# print("y_val shape:", y_val.shape)
# print("y_test shape:", y_test.shape)

In [5]:
# df_lab = pd.read_csv(attributes_path, sep="\s+", skiprows=1, usecols=['Male'])

# # Make 0 (female) & 1 (male) labels instead of -1 & 1
# df_lab.loc[df_lab['Male'] == -1, 'Male'] = 0

# df_lab.head()

In [6]:
# df_part = pd.read_csv(partitions_path, sep="\s+", skiprows=0, header=None)
# df_part.columns = ['Image_Name', 'Partition']
# df_part = df_part.set_index('Image_Name')
# print(df_part.head())

In [7]:
# df_lab_part = df_lab.merge(df_part, left_index=True, right_index=True)
# df_lab_part.index = df_lab_part.index.rename('Image_Name')
# # df6 = df_lab_part[:3]
# # df6.columns = ['Male', 'Partition']

# # print(df6)
# print(df_lab_part.head())


In [8]:
# df_race = pd.read_csv(race_path)
# df_race.columns = ['Image_Name', 'Race', 'Race_Scores']
# df_race = df_race.set_index('Image_Name')
# df_race = df_race['Race']
# print(df_race.head())

In [9]:
# df_total = df_lab_part.merge(df_race, left_index=True, right_index=True) #how="left",
# df_total.head()

In [10]:
# train_set = df_total.loc[df_total['Partition'] == 0][:size_train]
# df_lab_prepared = df_lab[:size_train].rename_axis("Image_Name").reset_index()
# train_set = pd.DataFrame(columns=df_lab_prepared.columns)
# # print(train_set)
# for i, row in tqdm(df_lab_prepared.iterrows(), total=size_train):
#     # print(i)
#     # img_number = i.split(".")[0]
#     img_number = f"{(i+1):06}"
#     for j, race in enumerate(races):
#         new_image_name = f"{img_number}_{j}_{race.lower()}.jpg"
#         row[0] = new_image_name
#         row["Race"] = race
#         train_set = train_set.append(row)
# print(train_set.shape)
# train_set.to_csv(label_dir + "gender_train_500.csv")



# val_set = df_total.loc[df_total['Partition'] == 1]
# test_set = df_total.loc[df_total['Partition'] == 2]

# print(len(train_set), len(val_set), len(test_set))
# print(val_set.head())
# train_set.to_csv('celeba-gender-train.csv')
# val_set.to_csv('celeba-gender-val.csv')
# test_set.to_csv('celeba-gender-test.csv')


In [11]:
# # split validation set by race
# grouped_df = val_set.groupby(val_set.Race)
# val_races = []
# print(f"Validation set ({len(val_set)} images):")
# for race in races:
#       val_race = grouped_df.get_group(race)
#       print(f"{race}: {len(val_race)}", end="\t")
#       val_race.to_csv(label_dir+"val/"+race+".csv")
# print("\n")

# # split test set by race
# grouped_df = test_set.groupby(test_set.Race)
# test_races = []
# print(f"Test set ({len(val_set)} images):")
# for race in races:
#       test_race = grouped_df.get_group(race)
#       print(f"{race}: {len(test_race)}", end="\t")
#       test_race.to_csv(label_dir+"test/"+race+".csv")

In [12]:
class CelebaDataset(Dataset):
    """Custom Dataset for loading CelebA face images"""

    def __init__(self, csv_path, img_dir, transform=None):
    
        df = pd.read_csv(csv_path, index_col=None)
        self.img_dir = img_dir
        self.csv_path = csv_path
        self.img_names = df["Image_Name"].values
        self.races = df["Race"].values
        self.y = df['Male'].values
        self.transform = transform

    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_dir,
                                      self.img_names[index]))
        
        if self.transform is not None:
            img = self.transform(img)
        
        label = self.y[index]
        gt_race = self.races[index]
        return img, label, gt_race

    def __len__(self):
        return self.y.shape[0]

In [13]:
custom_transform = transforms.Compose([transforms.Resize(FEATURE_SIZE),
                                       transforms.ToTensor()])

train_csv = "gender_train_500.csv" if use_baseline else "created_train.csv"
train_img_dir = img_celeb_dir if use_baseline else img_created_dir

train_dataset = CelebaDataset(csv_path=label_dir + train_csv,
                              img_dir=train_img_dir,
                              transform=custom_transform)

val_dataset_total = CelebaDataset(csv_path=label_dir + "gender_val.csv",
                              img_dir=img_celeb_dir,
                              transform=custom_transform)

test_dataset_total = CelebaDataset(csv_path=label_dir + "gender_test.csv",
                              img_dir=img_celeb_dir,
                              transform=custom_transform)

num_workers = 8
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE_TRAIN,
                          shuffle=True,
                          num_workers=num_workers)

val_loader_total = DataLoader(dataset=val_dataset_total,
                          batch_size=BATCH_SIZE_VAL,
                          shuffle=False,
                          num_workers=num_workers)

test_loader_total = DataLoader(dataset=test_dataset_total,
                          batch_size=BATCH_SIZE_TEST,
                          shuffle=False,
                          num_workers=16)


# split in 7 races

# val_loaders = []
# for race in races:
#     ds = CelebaDataset(csv_path=f"{label_dir}/val/{race}.csv",
#                         img_dir=img_celeb_dir,
#                         transform=custom_transform)
#     dl = DataLoader(dataset=ds,
#                     batch_size=BATCH_SIZE_VAL,
#                     shuffle=False,
#                     num_workers=num_workers)
#     val_loaders.append(dl)


# test_loaders = []
# for race in races:
#     ds = CelebaDataset(csv_path=f"{label_dir}/test/{race}.csv",
#                               img_dir=img_celeb_dir,
#                               transform=custom_transform)
#     dl = DataLoader(dataset=ds,
#                     batch_size=BATCH_SIZE_TEST,
#                     shuffle=False,
#                     num_workers=num_workers)
#     test_loaders.append(ds)

In [14]:
# part1 = pd.read_csv("race_total.csv")
# part2 = pd.read_csv("race_total_2.csv")
# part3 = pd.read_csv("race_total_3.csv")
# total = pd.concat([part1, part2, part3]).sort_values("face_name_align")
# total = total.set_index("face_name_align")
# print(total.head())
# total.to_csv("race_all.csv")

In [15]:
torch.manual_seed(RANDOM_SEED)

##########################
### COST AND OPTIMIZER
##########################


# model = resnet101(weights=ResNet101_Weights.DEFAULT)
model = resnet34(weights=ResNet34_Weights.DEFAULT)
model.to(DEVICE)
fc_layer = nn.Linear(1000, 1, device=DEVICE)
sigmoid = nn.Sigmoid()
tanh = nn.Tanh()
# softmax = nn.Softmax()
relu = nn.ReLU()
mse = nn.MSELoss()

bin_ce = nn.BCELoss()
# cost_fn = torch.nn.CrossEntropyLoss()  
params = list(model.parameters()) + list(fc_layer.parameters())
optimizer = torch.optim.Adam(params, lr=LEARNING_RATE)  

In [16]:
def get_elapsed_time(start_time):
    elapsed = int(time.time() - start_time)
    m, s = divmod(elapsed, 60)
    h, m = divmod(m, 60)
    return f"{h}:{m:02d}:{s:02d}"

In [17]:
# data_loader = val_loaders[6]

# correct_predictions = np.zeros(len(races))
# true_pos = np.zeros(len(races))
# true_neg = np.zeros(len(races))
# positive_preds = np.zeros(len(races))
# positive_targets = np.zeros(len(races))
# num_examples = np.zeros(len(races))
# total_examples = len(data_loader.dataset) 


# print("Evaluating...")
# total_it = int(np.ceil(total_examples / data_loader.batch_size))
# for i, (features, targets, gt_races) in tqdm(enumerate(data_loader), total=total_it):
#     prediction = np.random.randint(2, size=targets.shape)
#     targets = targets.numpy()
#     gt_races = np.array([races.index(race) for race in gt_races])
    
#     for j in range(len(races)):
#         correct_preds = (gt_races == j) & (prediction == targets)
#         true_pos[j] += (correct_preds & (prediction == 1)).sum()
#         true_neg[j] += (correct_preds & (prediction == 0)).sum()
#         correct_predictions[j] += correct_preds.sum()
#         positive_targets[j] += ((gt_races == j) & (targets == 1)).sum()
#         positive_preds[j] += np.where(gt_races == j, prediction, 0).sum()
#         num_examples[j] += (gt_races == j).sum()

#     # if i == 5:
#     #     break
# zero = 1e-10

# total_accuracy = correct_predictions.sum() / total_examples
# accuracies = [f"{a:.2%}" for a in correct_predictions / (num_examples + zero)]

# total_precision = true_pos.sum() / (positive_preds.sum() + zero)
# precisions = [f"{p:.2%}" for p in true_pos / (positive_preds + zero)]

# total_recall = true_pos.sum() / (positive_targets.sum() + zero)
# recalls = [f"{r:.2%}" for r in true_pos / (positive_targets + zero)]


# print(f"Total accuracy: {total_accuracy:.2%}\t| " +
#         f"Accuracies:\t{accuracies}\n")

# print(f"Total precision: {total_precision:.2%}\t| " +
#         f"Precisions:\t{precisions}\n")

# print(f"Total recall: {total_recall:.2%}\t| " +
#         f"Recalls:\t{recalls}")

In [18]:
# def compute_accuracy(model, data_loader, device):
#     correct_pred, num_examples = 0, 0
#     for i, (features, targets) in enumerate(data_loader):
#         if i == 5:
#             break
        
#         features = features.to(device)
#         targets = targets.to(device)
#         probas = sigmoid(fc_layer(model(features))).flatten()
#         prediction = probas >= 0.5
#         # print(prediction)
#         # print((prediction==targets).sum().float().item())
        
#         num_examples += targets.size(dim=0)
#         correct_pred += (prediction == targets).sum().float().item()
#     return correct_pred/num_examples * 100

def evaluate_metrics(model, data_loader, device, early_stop=-1):
    correct_predictions = np.zeros(len(races))
    true_pos = np.zeros(len(races))
    true_neg = np.zeros(len(races))
    positive_preds = np.zeros(len(races))
    positive_targets = np.zeros(len(races))
    num_examples = np.zeros(len(races))
    total_examples = len(data_loader.dataset) 

    total_it = int(np.ceil(total_examples / data_loader.batch_size))
    for i, (features, targets, gt_races) in tqdm(enumerate(data_loader), total=total_it, desc="Evaluating", disable=early_stop > 0):
        if i == early_stop:
            break

        features = features.to(device)
        probas = sigmoid(fc_layer(model(features))).flatten()
        prediction = (probas >= 0.5).cpu().numpy()
        targets = targets.numpy()
        gt_races = np.array([races.index(race) for race in gt_races])
        
        for j in range(len(races)):
            correct_preds = (gt_races == j) & (prediction == targets)
            true_pos[j] += (correct_preds & (prediction == 1)).sum()
            true_neg[j] += (correct_preds & (prediction == 0)).sum()
            correct_predictions[j] += correct_preds.sum()
            positive_targets[j] += ((gt_races == j) & (targets == 1)).sum()
            positive_preds[j] += np.where(gt_races == j, prediction, 0).sum()
            num_examples[j] += (gt_races == j).sum()

        
    zero = 1e-10

    total_accuracy = correct_predictions.sum() / num_examples.sum()
    accuracies = [f"{a:.2%}" for a in correct_predictions / (num_examples + zero)]

    total_precision = true_pos.sum() / (positive_preds.sum() + zero)
    precisions = [f"{p:.2%}" for p in true_pos / (positive_preds + zero)]

    total_recall = true_pos.sum() / (positive_targets.sum() + zero)
    recalls = [f"{r:.2%}" for r in true_pos / (positive_targets + zero)]
    return total_accuracy, accuracies, total_precision, precisions, total_recall, recalls

    

start_time = time.time()
    
batches_per_epoch = int(np.ceil((size_train * images_per_image_id) / BATCH_SIZE_TRAIN))


for epoch in range(NUM_EPOCHS):
    
    model.train()
    pbar = tqdm(enumerate(train_loader), total=batches_per_epoch, desc=f"Epoch {(epoch+1):02d}/{NUM_EPOCHS:02d}")
    for batch_idx, (features, targets, _) in pbar:
        
        features = features.to(DEVICE)
        targets = targets.float().to(DEVICE)
            
        ### FORWARD AND BACK PROP
        model_output = model(features)
        logits = sigmoid(fc_layer(model_output)).flatten()
        loss = bin_ce(logits, targets)
        pbar.set_postfix({"loss": f"{loss.item():.4f}"})

        optimizer.zero_grad()
        
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        # if batch_idx == 1:
        #     break 

    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        acc_total, accs, prec_total, precs, rec_total, recs = evaluate_metrics(model, val_loader_total, DEVICE, early_stop=20)
        print(f"Evaluation epoch {(epoch+1):02d}/{NUM_EPOCHS:02d}:")
        print(f"Total accuracy: {acc_total:.2%}\t| Accuracies:\t{accs}")
        print(f"Total precision: {prec_total:.2%}\t| Precisions:\t{precs}")
        print(f"Total recall: {rec_total:.2%}\t| Recalls:\t{recs}\n")
        # print(  f"Epoch: {(epoch+1):02d}/{NUM_EPOCHS:02d} | " + 
                #f"Train loss: {loss.item():.4f} |" +
                # f"Train Acc: {compute_accuracy(model, train_loader, device=DEVICE):.2f}% | " +
                # f"Val Acc: {compute_accuracy(model, val_loader_total, device=DEVICE):.2f}%") # | " + 
                # f"Val Acc: {compute_accuracy(model, val_loader_total, device=DEVICE):.2f}%") # | " + 
                #f"Time elapsed: {get_elapsed_time(start_time)}\n")
        # for i, race in enumerate(races):
        #     print(f"{race}: {compute_accuracy(model, val_loaders[i], device=DEVICE):.2f}%", end="\t")
        # print()
    
print(f"Total Training Time: {get_elapsed_time(start_time)}")

Epoch 01/10: 100%|██████████| 28/28 [00:39<00:00,  1.40s/it, loss=0.0946]


Evaluation epoch 01/10:
Total accuracy: 91.29%	| Accuracies:	['90.79%', '89.74%', '92.25%', '92.09%', '88.10%', '86.27%', '91.69%']
Total precision: 91.64%	| Precisions:	['90.18%', '100.00%', '88.64%', '95.97%', '87.50%', '86.57%', '91.76%']
Total recall: 88.31%	| Recalls:	['97.12%', '76.47%', '86.67%', '90.84%', '91.30%', '82.86%', '87.42%']



Epoch 02/10: 100%|██████████| 28/28 [00:36<00:00,  1.30s/it, loss=0.0419]


Evaluation epoch 02/10:
Total accuracy: 91.60%	| Accuracies:	['90.79%', '92.31%', '92.96%', '92.56%', '90.48%', '85.62%', '91.96%']
Total precision: 90.23%	| Precisions:	['90.18%', '93.75%', '84.31%', '96.75%', '91.30%', '84.29%', '90.01%']
Total recall: 90.79%	| Recalls:	['97.12%', '88.24%', '95.56%', '90.84%', '91.30%', '84.29%', '90.26%']



Epoch 03/10: 100%|██████████| 28/28 [00:36<00:00,  1.30s/it, loss=0.0016]


...

Evaluation epoch 08/10:
Total accuracy: 91.72%	| Accuracies:	['90.79%', '92.31%', '91.55%', '91.16%', '90.48%', '86.93%', '92.29%']
Total precision: 91.72%	| Precisions:	['90.91%', '93.75%', '85.11%', '96.67%', '91.30%', '87.88%', '91.77%']
Total recall: 89.28%	| Recalls:	['96.15%', '88.24%', '88.89%', '88.55%', '91.30%', '82.86%', '89.04%']



Epoch 09/10: 100%|██████████| 28/28 [00:36<00:00,  1.29s/it, loss=0.0027]


Evaluation epoch 09/10:
Total accuracy: 91.56%	| Accuracies:	['90.79%', '92.31%', '90.85%', '91.63%', '90.48%', '86.93%', '92.07%']
Total precision: 91.09%	| Precisions:	['90.18%', '93.75%', '83.33%', '96.69%', '91.30%', '87.88%', '91.03%']
Total recall: 89.64%	| Recalls:	['97.12%', '88.24%', '88.89%', '89.31%', '91.30%', '82.86%', '89.31%']



Epoch 10/10: 100%|██████████| 28/28 [00:36<00:00,  1.30s/it, loss=0.0030]


Evaluation epoch 10/10:
Total accuracy: 91.60%	| Accuracies:	['91.45%', '92.31%', '91.55%', '91.16%', '90.48%', '86.93%', '92.07%']
Total precision: 92.16%	| Precisions:	['91.74%', '93.75%', '86.67%', '96.67%', '91.30%', '87.88%', '92.20%']
Total recall: 88.49%	| Recalls:	['96.15%', '88.24%', '86.67%', '88.55%', '91.30%', '82.86%', '87.96%']

Total Training Time: 0:08:57


In [19]:
with torch.set_grad_enabled(False): # save memory during inference
    acc_total, accs, prec_total, precs, rec_total, recs = evaluate_metrics(model, test_loader_total, DEVICE)
    print(f"Evaluation test set:")
    print(f"Total accuracy: {acc_total:.2%}\t| Accuracies:\t{accs}")
    print(f"Total precision: {prec_total:.2%}\t| Precisions:\t{precs}")
    print(f"Total recall: {rec_total:.2%}\t| Recalls:\t{recs}\n")
    # print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader_total, device=DEVICE)))

    # for i, race in enumerate(races):
    #     print(f"{race}: {compute_accuracy(model, val_loaders[i], device=DEVICE):.2f}%", end="\t")
    # print()

Evaluating: 100%|██████████| 156/156 [01:29<00:00,  1.74it/s]

Evaluation test set:
Total accuracy: 91.94%	| Accuracies:	['86.52%', '93.67%', '94.64%', '92.00%', '84.24%', '90.88%', '92.53%']
Total precision: 89.26%	| Precisions:	['85.13%', '92.65%', '89.04%', '91.72%', '80.25%', '84.80%', '90.37%']
Total recall: 89.97%	| Recalls:	['92.41%', '90.43%', '91.62%', '91.84%', '87.50%', '87.46%', '89.49%']



In [20]:
# for batch_idx, (features, targets) in enumerate(test_loader_total):

#     features = features
#     targets = targets
#     break
    
# plt.imshow(np.transpose(features[0], (1, 2, 0)))

In [21]:
# model.eval()
# logits, probas = model(features.to(DEVICE)[0, None])
# print('Probability Female %.2f%%' % (probas[0][0]*100))